In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nplin
import itertools
from coniii import *

import timeit

In [2]:
np.random.seed(0)

In [3]:
def operators(s):
    #generate terms in the energy function
    n_seq,n_var = s.shape
    ops = np.zeros((n_seq,n_var+int(n_var*(n_var-1)/2.0)))

    jindex = 0
    for index in range(n_var):
        ops[:,jindex] = s[:,index]
        jindex +=1

    for index in range(n_var-1):
        for index1 in range(index+1,n_var):
            ops[:,jindex] = s[:,index]*s[:,index1]
            jindex +=1
            
    return ops

In [4]:
def energy_ops(ops,w):
    return np.sum(ops*w[np.newaxis,:],axis=1)

In [5]:
def generate_seqs(n_var,n_seq,n_sample=30,g=1.0):
    samples = np.random.choice([1.0,-1.0],size=(n_seq*n_sample,n_var),replace=True)
    ops = operators(samples)
    n_ops = ops.shape[1]

    #w_true = g*(np.random.rand(ops.shape[1])-0.5)/np.sqrt(float(n_var))
    w_true = np.random.normal(0.,g/np.sqrt(n_var),size=n_ops)

    sample_energy = energy_ops(ops,w_true)
    p = np.exp(sample_energy)
    p /= np.sum(p)
    out_samples = np.random.choice(np.arange(n_seq*n_sample),size=n_seq,replace=True,p=p)
    
    return w_true,samples[out_samples] #,p[out_samples],sample_energy[out_samples]

In [6]:
def eps_machine(s,eps_scale=0.1,max_iter=151,alpha=0.1):
    MSE = np.zeros(max_iter)
    KL = np.zeros(max_iter)
    E_av = np.zeros(max_iter)
    
    n_seq,n_var = s.shape
    ops = operators(s)
    n_ops = ops.shape[1]
    cov_inv = np.eye(ops.shape[1])

    np.random.seed(13)
    w = np.random.rand(n_ops)-0.5    
    
    w_iter = np.zeros((max_iter,n_ops))
    for i in range(max_iter): 
        
        eps_scale = np.random.rand()/np.max([1.,np.max(np.abs(w))])
        
        energies_w = energy_ops(ops,w)
        probs_w = np.exp(-energies_w*(1-eps_scale))
        z_data = np.sum(probs_w)
        probs_w /= z_data
        ops_expect_w = np.sum(probs_w[:,np.newaxis]*ops,axis=0)
        
        #if iterate%int(max_iter/5.0)==0:
        E_exp = (probs_w*energies_w).sum()
        KL[i] = -E_exp - np.log(z_data) + np.sum(np.log(np.cosh(w*eps_scale))) + n_var*np.log(2.)
        
        E_av[i] = energies_w.mean()
        MSE[i] = ((w-w_true)**2).mean()
        #print(RMSE[i])
        #print(eps_scale,iterate,nplin.norm(w-w_true),RMSE,KL,E_av)
        sec_order = w*eps_scale
        w += alpha*cov_inv.dot((ops_expect_w - sec_order))        
        #print('final ',eps_scale,iterate,nplin.norm(w-w_true))    
        w_iter[i,:] = w
        
    return MSE,KL,E_av,w_iter

In [7]:
def hopfield_model(s):
    ops = operators(s)
    w = np.mean(ops,axis=0)
    #print('hopfield error ',nplin.norm(w-w_true))
    return w

In [8]:
def boltzmann_machine_exact(s,s_all,max_iter=150,alpha=5e-2,cov=False):
    n_seq,n_var = s.shape
    ops = operators(s)
    cov_inv = np.eye(ops.shape[1])
    ops_obs = np.mean(ops,axis=0)
    ops_model = operators(s_all)

    n_ops = ops.shape[1]
    
    np.random.seed(13)
    w = np.random.rand(n_ops)-0.5    
    for iterate in range(max_iter):
        energies_w = energy_ops(ops_model,w)
        probs_w = np.exp(energies_w)
        probs_w /= np.sum(probs_w)
        if iterate%10 == 0: 
            #print(iterate,nplin.norm(w-w_true)) #,nplin.norm(spin_cov_w-spin_cov_obs))
            MSE = ((w-w_true)**2).mean()
            print(iterate,MSE)
            
        w += alpha*cov_inv.dot(ops_obs - np.sum(ops_model*probs_w[:,np.newaxis],axis=0))

    print('final',iterate,MSE)

    return w

In [9]:
max_iter = 151

n_seq = 10000
g = 1.

nn_var = 5
n_var_list = np.linspace(42,50,nn_var).astype(int)
n_var_list

array([42, 44, 46, 48, 50])

In [10]:
run_time = np.zeros(nn_var)

for i,n_var in enumerate(n_var_list):    
    n_var = int(n_var)

    w_true,seqs = generate_seqs(n_var,n_seq,g=g)

    np.random.seed(13)

    start_time = timeit.default_timer()

    # Define common functions
    calc_e,calc_observables,mchApproximation = define_ising_helper_functions()    
    get_multipliers_r,calc_observables_r = define_pseudo_ising_helpers(n_var)

    solver = Pseudo(n_var,calc_observables=calc_observables,
                    calc_observables_r=calc_observables_r,
                    get_multipliers_r=get_multipliers_r)

    w_pseudo = solver.solve(seqs,np.zeros(n_var+int(n_var*(n_var-1)/2.0)))

    MSE_pseudo = ((w_true - w_pseudo)**2).mean()

    run_time[i] = timeit.default_timer() - start_time
    print('run_time:',n_var,run_time[i],MSE_pseudo)

/home/tai/anaconda3/lib/python3.7/site-packages/coniii/utils.py:696: UserWarning: DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions
  warn("DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions")


run_time: 42 402.8438947269897 0.014639483668923994


/home/tai/anaconda3/lib/python3.7/site-packages/coniii/utils.py:696: UserWarning: DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions
  warn("DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions")


run_time: 44 434.2705662060034 0.014953270662897478


/home/tai/anaconda3/lib/python3.7/site-packages/coniii/utils.py:696: UserWarning: DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions
  warn("DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions")


run_time: 46 851.8924601229955 0.041730422025451394


/home/tai/anaconda3/lib/python3.7/site-packages/coniii/utils.py:696: UserWarning: DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions
  warn("DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions")


run_time: 48 500.31253179399937 0.03258593248389198


/home/tai/anaconda3/lib/python3.7/site-packages/coniii/utils.py:696: UserWarning: DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions
  warn("DEPRECATION WARNING: now renamed to define_pseudo_ising_helper_functions")


run_time: 50 454.40339095199306 0.01929571421585285


In [11]:
np.savetxt('run_time_pseudo_m42-50.txt',(n_var_list,run_time),fmt='%f')